### Uso de Structured streaming.

Se recogen los datos en streaming desde Kafka, y se les aplicará el análisis de sentimiento para etiquetar el sentimiento de cada tweet recogido.

In [1]:
# imports necesarios
import pandas as pd

# Spark Streaming
from pyspark.streaming import StreamingContext  
# Kafka
from pyspark.streaming.kafka import KafkaUtils

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import *
from pyspark.sql.functions import *

from pymongo import MongoClient

!pip install elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers

import nltk
from nltk.tokenize import word_tokenize
from nltk import TweetTokenizer
from nltk.corpus import stopwords
# cargamos las stopwords para cada idioma
spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')

import json
import string
import re
import unicodedata

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


spark = SparkSession\
          .builder\
          .appName("twitter")\
          .master("spark://MacBook-Pro-de-Jose.local:7077")\
          .config("spark.io.compression.codec", "snappy")\
          .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

#### Creamos las funciones necesarias.

In [2]:
# Función de carga de datos desde MongoDB de los datos almacenados desde Apache Nifi, tanto en inglés como español
def carga_datos_mongo():
    # conectamos con las tablas de Mongo desde donde cargamos los datos
    df_mongo_english = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
        .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_english").load()
    df_mongo_spanish = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
        .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_spanish").load()
    
    # nos quedamos solo con el atributo texto que es la información que nos interesa de cada dataset
    df_text_english = df_mongo_english[['text']]
    df_text_spanish = df_mongo_spanish[['text']]
    
    # pasamos a DF pandas
    df_pandas_english = df_text_english.toPandas()
    df_pandas_spanish = df_text_spanish.toPandas()
    
    return df_pandas_english, df_pandas_spanish


# Función de carga de datos desde los csv generados con anterioridad con registros ya con el sentimiento anotado
def carga_datos_csv(csv):
    # cargamos el csv
    if(csv=='english_full'):
        df_anotados = pd.read_csv('./data/df_result_english.csv', sep=',')
    elif(csv=='spanish_full'):
        df_anotados = pd.read_csv('./data/df_result_spanish.csv', sep=',')
    elif(csv=='english_neutro'):
        df_anotados = pd.read_csv('./data/df_result_english_neutral.csv', sep=',')
    elif(csv=='spanish_neutro'):
        df_anotados = pd.read_csv('./data/df_result_spanish_neutral.csv', sep=',')
    elif(csv=='english_noNeutro'):
        df_anotados = pd.read_csv('./data/df_result_english_noNeutral.csv', sep=',')
    elif(csv=='spanish_noNeutro'):
        df_anotados = pd.read_csv('./data/df_result_spanish_noNeutral.csv', sep=',')
        
    # eliminamos posibles valores de sentimiento vacio, pasándo el valor a -1 y eliminando el registro después
    df_anotados = df_anotados.fillna(-1)
    df_anotados = df_anotados[df_anotados['sentiment']!=-1]
    
    # el atributo sentimiento nos aseguramos que sea tipo int en lugar de float
    col_float = df_anotados.columns[df_anotados.dtypes == float]
    df_anotados[col_float] = df_anotados[col_float].astype(int)

    # como los datos en inglés del df full son muy grandes, para evitar problemas computacionales nos quedamos 
    # con una parte similar al tamaño de los datos que tenemos en español
    if(csv=='english_full'):
        df_anotados = df_anotados.sample(frac=0.04, replace=False, random_state=1)
    
    return df_anotados


# Función de visualización de DFs: número de registros y columnas, lista de columnas, recuento de los distintos
# valores de la columna sentimiento.
def visualizar_datos_df(df,tipo):
    print("num_rows: %d\tColumnas: %d\n" % (df.shape[0], df.shape[1]) )
    print("Columnas:\n", list(df.columns))
    print("\n")
    if(tipo==0):
        print("Recuento valores columna sentimiento:\n", pd.value_counts(df['sentiment']))
        print("\n")


# Función de limpieza de los tweets
def limpiar_tweet(tweet):
    # quitamos RT, @nombre_usuario, links y urls, hashtags, menciones, caracteres extraños o emoticonos
    tweet = re.sub('  +', ' ', tweet)
    # eliminar acentos
    tweet = ''.join((c for c in unicodedata.normalize('NFD', tweet) if unicodedata.category(c) != 'Mn'))  
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)
    # eliminar "RT", "@usuario", o los enlaces que es información que no sería útil analizar
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub('','',tweet).lower() 
    tweet = re.sub(r'http\S+', '', tweet) 
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet)
    tweet = re.sub(r'[0-9]', '', tweet) 

    return tweet 


# Función que le pasamos un Dataframe y nos lo devuelve con los datos preprocesados con la función anterior
def limpiar_df(df):
    df_clean = df.copy()
    df_clean['text'] = df_clean['text'].apply(limpiar_tweet)
    
    return df_clean


# Función que genera los conjuntos de train y test a partir de un dataframe. Reparto 90% train - 10% test
def split_df(df):
    X_eng = df['text']
    y_eng = df['sentiment']

    X, X_test, y, y_test = train_test_split(X_eng, y_eng, test_size=0.1, random_state=42)

    return X, X_test, y, y_test


# Función que genera el pipeline y parámetros del modelo, y lo entrena.
# Se centra la prueba en ver el resultado con distintos clasificadores para ver que tipo de algoritmo da los 
# mejores resultados. En cuanto al vectorizador se prueba tanto con CountVectorizer como con TDIDFVectorizer.
# No se definen muchos otros parámetros por cuestión de tiempo y del elevado coste computacional en el caso de
# jugar con muchos parámetros y varias posibilidades para cada uno.
def generar_modelo(clasificador, stopwords, X_train, y_train):
    # uso de tokenizer que es una buena práctica en el tratamiento de textos
    tokenizer = TweetTokenizer().tokenize

    pipeline = Pipeline([
        ('vectorizer', None),
        ('classifier', clasificador)]
    )
    
    # también se prueba a usar o no stopwords, que serían palabras muy comunes en el lenguaje, que no tienen valor
    # real a la hora de analizar el sentimiento de un texto y pueden no tenerse en cuenta.
    params = {
        'vectorizer': [CountVectorizer(binary=True,tokenizer=tokenizer),\
                       CountVectorizer(binary=False,tokenizer=tokenizer),\
                       TfidfVectorizer(use_idf=False, tokenizer=tokenizer),\
                       TfidfVectorizer(use_idf=True, tokenizer=tokenizer)],
        'vectorizer__stop_words': [None, stopwords],
    }

    # uso también de StratifiedKFold que es otra buena práctica
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    modelo = GridSearchCV(pipeline, params, n_jobs=-1, cv=skf, refit='f1_weighted')
    modelo.fit(X_train, y_train)

    return modelo


# Función que evalua el modelo que le pasamos y saca las métricas que se quieren observar
def evaluar_modelos(modelo1, modelo2, x_eng_test, x_spa_test, y_eng_test, y_spa_test):
    # pintamos los mejores parámetros de cada modelo
    print("Mejores parámetros del modelo para datos en inglés: \n", modelo1.best_params_)
    print("\n")
    print("Mejores parámetros del modelo para datos en español: \n", modelo2.best_params_)

    # se usan los modelos entrenados para predecir los datos de test
    y_pred_eng = modelo1.predict(x_eng_test)
    y_pred_spa = modelo2.predict(x_spa_test)

    # pintamos las métricas de cada modelo
    print("\n")
    print("Métricas del modelo con datos en inglés:\n")
    print(classification_report(y_eng_test, y_pred_eng, target_names=None))
    print("\n")
    print("Métricas del modelo con datos en español:\n")
    print(classification_report(y_spa_test, y_pred_spa, target_names=None))



#### Carga de los datos guardados en formato .csv para entrenar el modelo a usar con los tweets que vienen en streaming.

In [3]:
# cargamos los distintos csv generados con anterioridad y con el sentimiento ya etiquetado
df_csv_english_full = carga_datos_csv('english_full')
df_csv_spanish_full = carga_datos_csv('spanish_full')
df_csv_english_neutro = carga_datos_csv('english_neutro')
df_csv_spanish_neutro = carga_datos_csv('spanish_neutro')
df_csv_english_noNeutro = carga_datos_csv('english_noNeutro')
df_csv_spanish_noNeutro = carga_datos_csv('spanish_noNeutro')

In [4]:
# de momento usaremos los csv full con todos los datos
df_csv_english_full.head(10)

,text,sentiment
128038,@BradshawPhotogr seems i always end up at tx ...,2
491756,"@unitechy yeah, don't worry, you will!!! there...",0
470925,Excited that my email works reliably now with ...,0
491264,@MF213 yea that's the sad part,0
836490,@modeladrienne you let me know! I think you to...,0
371404,2155 - Well - Susan didn't make it ... Runne...,0
73350,..Before the cool runs out..Ima be trying my B...,2
1166160,@dbdc LMAO not today sir sorry sir I did go ye...,2
1070017,LOVES that lubbock is wet..its about time..no ...,2
229521,I want a golden retriever puppy!! soooo cute!!...,0


In [5]:
df_csv_spanish_full.head(10)

,text,sentiment
0,@PauladeLasHeras No te libraras de ayudar me/n...,1
1,@marodriguezb Gracias MAR,2
2,"Off pensando en el regalito Sinde, la que se v...",0
3,Conozco a alguien q es adicto al drama! Ja ja ...,2
4,Toca @crackoviadeTV3 . Grabación dl especial N...,2
5,Buen día todos! Lo primero mandar un abrazo gr...,2
6,Desde el escaño. Todo listo para empezar #endi...,2
7,Bdías. EM no se ira de puente. Si vosotros os ...,2
8,Un sistema económico q recorta dinero para pre...,2
9,#programascambiados caca d ajuste,0


#### Generamos el modelo y lo entrenamos con los datos recogidos de los csv

In [6]:
# se limpian los tweets antes de entrenar y evaluar el modelo
df_clean_english_full = limpiar_df(df_csv_english_full)
df_clean_spanish_full = limpiar_df(df_csv_spanish_full)

In [7]:
df_clean_english_full.head(10)

,text,sentiment
128038,seems i always end up at tx schl email lori...,2
491756,yeah don t worry you will there s still ...,0
470925,excited that my email works reliably now with ...,0
491264,yea that s the sad part,0
836490,you let me know i think you too busy for li...,0
371404,well susan didn t make it runner up to...,0
73350,before the cool runs out ima be trying my b...,2
1166160,lmao not today sir sorry sir i did go yester...,2
1070017,loves that lubbock is wet its about time no ...,2
229521,i want a golden retriever puppy soo cute d...,0


In [8]:
df_clean_spanish_full.head(10)

,text,sentiment
0,no te libraras de ayudar me nos besos y gra...,1
1,gracias mar,2
2,off pensando en el regalito sinde la que se v...,0
3,conozco a alguien q es adicto al drama ja ja ...,2
4,toca grabacion dl especial navideno mari ...,2
5,buen dia todos lo primero mandar un abrazo gr...,2
6,desde el escano todo listo para empezar endia...,2
7,bdias em no se ira de puente si vosotros os ...,2
8,un sistema economico q recorta dinero para pre...,2
9,programascambiados caca d ajuste,0


In [9]:
# se generan los conjuntos de datos para train y test
X_eng_train_full, X_eng_test_full, y_eng_train_full, y_eng_test_full = split_df(df_clean_english_full)
X_spa_train_full, X_spa_test_full, y_spa_train_full, y_spa_test_full = split_df(df_clean_spanish_full)

In [10]:
# generamos los modelos para datos en inglés y en español
clasificador = LinearSVC()

model_eng_svms = generar_modelo(clasificador, english_stopwords, X_eng_train_full, y_eng_train_full)
model_spa_svms = generar_modelo(clasificador, spanish_stopwords, X_spa_train_full, y_spa_train_full)

In [11]:
# evaluamos los modelos
evaluar_modelos(model_eng_svms, model_spa_svms, X_eng_test_full, X_spa_test_full, y_eng_test_full, y_spa_test_full)

Mejores parámetros del modelo para datos en inglés: 
 {'vectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x1a2d93ca58>>,
        use_idf=False, vocabulary=None), 'vectorizer__stop_words': None}


Mejores parámetros del modelo para datos en español: 
 {'vectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smoot

#### Conectamos con el streaming para recoger los datos de Kafka y tenerlos en un dataframe.

In [12]:
# definimos el schema de los datos que queremos guardar de los datos de entrada al sistema
schema = StructType([ 
    StructField("text", StringType(), True),
    StructField("sentiment", IntegerType(), True)
])

In [13]:
# se obtienen desde Kafka los datos del topic en español
df_spanish = spark\
.readStream\
.format("kafka")\
.option("kafka.bootstrap.servers", "localhost:9092")\
.option("startingOffsets", "earliest")\
.option("subscribe", "TopicSpanish")\
.option("failOnDataLoss", "false") \
.load()

df_spanish = df_spanish.selectExpr("CAST(value AS STRING)")

df_spanish = df_spanish.select(from_json(col("value"), schema).alias("data")).select("data.*")

df_spanish = df_spanish.filter(df_spanish.text.isNotNull())

df_spanish.createOrReplaceTempView("spanish")

In [14]:
# se obtienen desde Kafka los datos del topic en inglés
df_english = spark\
.readStream\
.format("kafka")\
.option("kafka.bootstrap.servers", "localhost:9093")\
.option("startingOffsets", "earliest")\
.option("subscribe", "TopicEnglish")\
.option("failOnDataLoss", "false") \
.load()

df_english = df_english.selectExpr("CAST(value AS STRING)")

df_english = df_english.select(from_json(col("value"), schema).alias("data")).select("data.*")

df_english = df_english.filter(df_english.text.isNotNull())

df_english.createOrReplaceTempView("english")

In [15]:
# chequeamos los datos en streaming
print(" ")
print("Is the stream ready?")
print(df_spanish.isStreaming, df_english.isStreaming)

 
Is the stream ready?
True True


In [20]:
# definimos querys en memoria para ver los datos que se han recogido
query_spanish = df_spanish.writeStream.outputMode("update").queryName("spanish").format("memory")\
.option("truncate", "False").start()

query_english = df_english.writeStream.outputMode("update").queryName("english").format("memory")\
.option("truncate", "False").start()

print(query_spanish.status)
print(query_english.status)

{'message': 'Getting offsets from KafkaV2[Subscribe[TopicSpanish]]', 'isDataAvailable': False, 'isTriggerActive': True}
{'message': 'Getting offsets from KafkaV2[Subscribe[TopicEnglish]]', 'isDataAvailable': False, 'isTriggerActive': True}


In [21]:
# pasamos a DF pandas
result_spanish = spark.table("spanish").toPandas()
result_english = spark.table("english").toPandas()

In [22]:
# visualizamos los datos en español
result_spanish

,text,sentiment
0,Jajajajajajaja https://t.co/zt4NncmOgg,None
1,"RT @00INVICTUS: Pleno al quince, todo el gobie...",None
2,RT @angelriver01: Hola mi nombre es Ángel Rive...,None
3,RT @SofaVallejos3: Abro hilo de las cosas favo...,None
4,RT @conchetujade: podemos hablar de el poder q...,None
5,RT @anluma99: La orquesta de Filadelfia y la Ó...,None
6,RT @LesbiQueen: SaLVeMoz LoZ DoZ LadRiLlOz htt...,None
7,"RT @glowingbell: Hola,estan las inscripciones ...",None
8,RT @FansVanesa_sexy: sin palabras ante tanta d...,None
9,Luis Zapata quedamos \n#PartyChilensisFtAndrei...,None


In [23]:
# visualizamos los datos en inglés
result_english

,text,sentiment
0,RT @profwolff: Local Maryland residents stand ...,None
1,@BloodyElbow I love Jose. He’s brought me so...,None
2,RT @JohnNosta: How do we manage “patients righ...,None
3,"RT @dwikaputra: In life, \npeople stayed for a...",None
4,RT @EpochTimes: “[#GhislaineMaxwell is] going ...,None
5,RT @pikaole: 🐦 Hummingbird 🎶 https://t.co/SapS...,None
6,RT @kimairapossible: True. The distance from o...,None
7,RT @ffgonzz: This is my shit. I need affection...,None
8,Antioxidants : any substance that reduces oxid...,None
9,"RT @once_tim: GB banner today: ""This is how it...",None


#### Ahora aplicamos a los DFs recogidos en streaming el modelo entrenado para etiquetar el sentimiento de cada tweet.

Vamos también a probar a limpiar estos datos, y aplicamos el modelo para predecir el sentimiento tanto a los datos en crudo como a los datos en limpio.

In [24]:
# hacemos una copia de los DFs en inglés y español
df_clean_spanish = limpiar_df(result_spanish)
df_clean_english = limpiar_df(result_english)

In [25]:
df_clean_spanish.head(10)

,text,sentiment
0,jajajajajajaja,None
1,pleno al quince todo el gobierno de gallar...,None
2,hola mi nombre es angel rivero me gradue de...,None
3,abro hilo de las cosas favoritas de namjoon...,None
4,podemos hablar de el poder que se cargan es...,None
5,la orquesta de filadelfia y la opera de san...,None
6,salvemoz loz doz ladrilloz,None
7,hola estan las inscripciones abiertas para ...,None
8,sexy sin palabras ante tanta delicia sexx...,None
9,luis zapata quedamos partychilensisftandreiha...,None


In [26]:
df_clean_english.head(10)

,text,sentiment
0,local maryland residents stand up fight lo...,None
1,i love jose he s brought me some of my grea...,None
2,how do we manage patients rights against ...,None
3,in life people stayed for a reason and ...,None
4,ghislainemaxwell is going down she s go...,None
5,hummingbird,None
6,true the distance from our bed to the kitc...,None
7,this is my shit i need affection,None
8,antioxidants any substance that reduces oxid...,None
9,tim gb banner today this is how it feels ...,None


In [27]:
# usamos el modelo para predecir el sentimiento de los tweets en los distintos DFs, por idioma y si están en crudo
# o no los datos.
y_pred_spa_clean = model_spa_svms.predict(df_clean_spanish.text)
y_pred_spa = model_spa_svms.predict(result_spanish.text)
y_pred_eng_clean = model_eng_svms.predict(df_clean_english.text)
y_pred_eng = model_eng_svms.predict(result_english.text)

In [28]:
# hacemos una copia de los DFs, y a la columna sentimiento le otorgamos los valores obtenidos con el modelo
df_predict_english = result_english.copy()
df_predict_english_clean = df_clean_english.copy()
df_predict_spanish = result_spanish.copy()
df_predict_spanish_clean = df_clean_spanish.copy()

df_predict_english['sentiment'] = y_pred_eng
df_predict_english_clean['sentiment'] = y_pred_eng_clean
df_predict_spanish['sentiment'] = y_pred_spa
df_predict_spanish_clean['sentiment'] = y_pred_spa_clean

In [29]:
# vamos a dar un vistazo a los primeros registros de cada DF para comparar el sentimiento calculado en cada caso
df_predict_english.head(20)

,text,sentiment
0,RT @profwolff: Local Maryland residents stand ...,2
1,@BloodyElbow I love Jose. He’s brought me so...,2
2,RT @JohnNosta: How do we manage “patients righ...,0
3,"RT @dwikaputra: In life, \npeople stayed for a...",0
4,RT @EpochTimes: “[#GhislaineMaxwell is] going ...,0
5,RT @pikaole: 🐦 Hummingbird 🎶 https://t.co/SapS...,2
6,RT @kimairapossible: True. The distance from o...,0
7,RT @ffgonzz: This is my shit. I need affection...,0
8,Antioxidants : any substance that reduces oxid...,0
9,"RT @once_tim: GB banner today: ""This is how it...",0


In [30]:
df_predict_english_clean.head(20)

,text,sentiment
0,local maryland residents stand up fight lo...,0
1,i love jose he s brought me some of my grea...,2
2,how do we manage patients rights against ...,0
3,in life people stayed for a reason and ...,0
4,ghislainemaxwell is going down she s go...,0
5,hummingbird,2
6,true the distance from our bed to the kitc...,0
7,this is my shit i need affection,0
8,antioxidants any substance that reduces oxid...,0
9,tim gb banner today this is how it feels ...,0


In [31]:
df_predict_spanish.head(20)

,text,sentiment
0,Jajajajajajaja https://t.co/zt4NncmOgg,2
1,"RT @00INVICTUS: Pleno al quince, todo el gobie...",0
2,RT @angelriver01: Hola mi nombre es Ángel Rive...,2
3,RT @SofaVallejos3: Abro hilo de las cosas favo...,2
4,RT @conchetujade: podemos hablar de el poder q...,0
5,RT @anluma99: La orquesta de Filadelfia y la Ó...,2
6,RT @LesbiQueen: SaLVeMoz LoZ DoZ LadRiLlOz htt...,2
7,"RT @glowingbell: Hola,estan las inscripciones ...",2
8,RT @FansVanesa_sexy: sin palabras ante tanta d...,2
9,Luis Zapata quedamos \n#PartyChilensisFtAndrei...,2


In [32]:
df_predict_spanish_clean.head(20)

,text,sentiment
0,jajajajajajaja,2
1,pleno al quince todo el gobierno de gallar...,0
2,hola mi nombre es angel rivero me gradue de...,0
3,abro hilo de las cosas favoritas de namjoon...,2
4,podemos hablar de el poder que se cargan es...,0
5,la orquesta de filadelfia y la opera de san...,2
6,salvemoz loz doz ladrilloz,2
7,hola estan las inscripciones abiertas para ...,2
8,sexy sin palabras ante tanta delicia sexx...,2
9,luis zapata quedamos partychilensisftandreiha...,2


Se aprecia como a simple vista hay muy pocos tweets que el modelo etiquete de forma distinta ya sea pasándole los datos en crudo para predecir como si primero se pasan por el procesado de limpieza de los textos.

In [33]:
# end streaming
query_spanish.stop()
query_english.stop()

#### Almacenar datos en CSV, MongoDB y ElasticSearch.

Probamos a almacenar los datos etiquetados en diferentes formatos: archivos csv, MongoDB, y en ElasticSearch para su posterior visualización con Kibana.

In [34]:
# sacamos una cadena con fecha y hora para añadirla al nombre de los csv que se generan a continuación
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
print(timestr)

20190818-133840


In [36]:
# guardar datos en csv
df_predict_spanish.to_csv('./data/predict_streaming_spanish_'+timestr+'.csv', index=False)
df_predict_english.to_csv('./data/predict_streaming_english_'+timestr+'.csv', index=False)

In [39]:
# guardar datos en MongoDB
conexion = 'mongodb://localhost:27017'
client = MongoClient(conexion)

# accedemos a la base de datos
db = client.tfm_twitter
# insertamos los dataframes en la tabla correspondiente
db.tweets_streaming_spanish.insert_many(df_predict_spanish.to_dict("records"))
db.tweets_streaming_english.insert_many(df_predict_english.to_dict("records"))

In [41]:
# guardar datos en ElasticSearch
es = Elasticsearch('http://localhost:9200/')

# generación de índices
!curl -X PUT "localhost:9200/tweets_sentiment_spa"
!curl -X PUT "localhost:9200/tweets_sentiment_eng"

INDEX_SPA = "tweets_sentiment_spa"
INDEX_ENG = "tweets_sentiment_eng"
TYPE = "record"

def rec_to_actions(df, lang):
    for record in df.to_dict(orient="records"):
        if(lang==0): yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX_SPA, TYPE))
        elif(lang==1): yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX_ENG, TYPE))
        yield (json.dumps(record, default=int))

if not es.indices.exists(INDEX_SPA):
    raise RuntimeError('index does not exists, use `curl -X PUT "localhost:9200/%s"` and try again'%INDEX_SPA)
if not es.indices.exists(INDEX_ENG):
    raise RuntimeError('index does not exists, use `curl -X PUT "localhost:9200/%s"` and try again'%INDEX_ENG)

r_spa = es.bulk(rec_to_actions(df_predict_spanish, 0))
r_eng = es.bulk(rec_to_actions(df_predict_english, 1)) 

print(not r_spa["errors"], not r_eng["errors"])

{"acknowledged":true,"shards_acknowledged":true,"index":"tweets_sentiment_spa"}{"acknowledged":true,"shards_acknowledged":true,"index":"tweets_sentiment_eng"}True True


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
